# Task 1: Parsing Text Files

**Name: Manmeet Singh**<br>
**Student Id: 30749476**

**Description**<br>
This assessment touches the very first step of analyzing textual data, i.e., extracting data from semi-structured text files. Data-set contains information about COVID-19 related tweets.<br>

**Steps to perform:**<br>
1. First step of analyzing textual data, i.e., extracting data from semi-structured text files.
2. Create XML file with following elements:<br>
    a. id: is a 19-digit number.<br>
    b. text: is the actual tweet.<br>
    c. Created_at: is the date and time that the tweet was created<br>
3. Id's must ne unique, filter out non-english tweets.<br>
4. Designing efficient regular expressions in order to extract the data from your dataset.<br>

## Importing the required Libraries

In [1]:
# importing required libraries
import re
import langid
import os
from langid.langid import LanguageIdentifier, model

## Method used to extract data

### Steps performed to extract the data<br>
1. Creating 2 dictionaries, one to store the final data and a temporary dictionary to store Id and text as key value.<br>
2. Creating a language identifier, as we load text file in UTF-8 format, file contains emojis as well. So, while clasifying the language there might be an error. Language identifier will identify language even if text contains emojis.<br>
3. Creating a list of documents to read.<br>
4. Opening each file one by one and performing following steps respectively:<br>
    a. Split data first based on regex **(r'\[(.*)\]')**. As our data is in format of dictionary in list, it will extract data inside the square braces. It will extract all the data inside the keyword "data". It will generate another list, containing tweet data and error logs.<br>
    b. Second regex **(r'\{"(.*?)\"}')** is used to extract the data inside the curly braces based on non-greedy method, as it is possible there maybe some text inside curly braces for "text" key. This will create another list. Each list is package of id, text and created_at data.<br>
    c. Checking if withheld, copyright, and country_codes exist while looping over the list created, as it is not part of tweets and might casue errors such as string literal or key word error for "false". If exist skip and go to next element in list.<br>
    d. Check every element of list if it contains all three terms text, id, created_at because error logs does not have text.<br>
    e. If given term exist add prefix and suffix of curly braces.<br>
    f. Use eval function as it will generate dictioinary from string of dictionary.<br>
    g. Extract the tweet from key "text".
    h. Convert the tweet to UTF-16 format as tweet might contain emojis as UTF-8 cannot comprehend it.<br>
    i. Filtering out the non-english tweets.<br>
    j. Using regex **(r'[0-9]+-[0-9]+-[0-9]+')** to find the date as it is in UTC format.<br>
    k. Adding date as key in NewDict (dictionary). If date already exist then update its value by id:text as key:value.<br>
    l. To avoid the duplicate id's check if id exist in value of date key. If exist skip else update to the date value.<br>
    m. Steps from a to l will be performed in loop and all the data will be added to NewDict where Date is key and id:text is value.<br>
    
    
    

In [2]:
# initiating temprory dictionary to store Id as key and text as value
temp = {}
# initiating New dictionary to store date as key and Id, text as value in dictionary format 
NewDict = {}
# creating a language identifier, as we load text file in UTF-8 format, file contains 
identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)

# creating a list of all the documents on the given path
all_files = os.listdir("30749476/")

# creating a for loop to call files one by one, reading them and processing them
for texFile in all_files:
    path = '30749476/{}'.format(texFile)
    
    with open(path, encoding = 'utf-8') as file:
            data = file.read()
    
    # Creating frist split of data using the regex
    tweetSplit1 = re.split(r'\[(.*)\]', data)
    # finding all the data as per the given regex
    tweetSplit2 = re.split(r'\{"(.*?)\"}',tweetSplit1[1])
    
    # reading all the data created by tweetSplit2
    for terms in tweetSplit2:
        # to avoid string literal and keyword error, a specific format contaning given terms is ignored
        if 'withheld' in terms and 'copyright' in terms and 'country_codes' in terms:
            continue
        if ("created_at" in terms) and ("text" in terms) and ("id" in terms):  # if text contain id, text and created_at terms then process it.
            # padding curly braces to create a dictionary format
            dictString = '{"' + terms + '"}'
            # calling eval function to generate dictionary from string of dictionary
            content = eval(dictString)
            # fetching text from the dictionary
            text = content["text"]
            # converting text to UTF-16 format to read emojis as well
            utf16 = text.encode('utf-16', 'surrogatepass').decode('utf-16')
            # if text is english then only process it
            if identifier.classify(utf16)[0] == 'en':
                # fetching date from the dictionary
                toMatch = content["created_at"]
                date = re.findall(r'[0-9]+-[0-9]+-[0-9]+',toMatch)[0]
                # if date does not exist in dictionary create one
                if date not in NewDict:
                    NewDict[date] = {}
                    # removes duplicate entry of id's
                    if content["id"] in NewDict[date]:
                        continue
                    temp[content["id"]] = content["text"] # creating key value pair of id and text
                    NewDict[date].update(temp) # adding to required date
                    temp = {}
                else:
                    if content["id"] in NewDict[date]: # removes duplicate entry of id's
                        continue
                    temp[content["id"]] = content["text"] # creating key value pair of id and text
                    NewDict[date].update(temp) # adding to required date
                    temp = {}

## Writing the data to XML file

In [3]:
with open('30749476.xml','w+', encoding='utf-8') as writeData: # write data in xml format
    writeData.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    writeData.write('<data>\n')
    
    for date, idText in NewDict.items(): 
        writeData.write('<tweets date="{}">\n'.format(date))
        
        for ID, Text in idText.items():
            Text = Text.encode('utf-16', 'surrogatepass').decode('utf-16') # decoding the text from UTF-16 again as encoding is UTF-8
            writeData.write('<tweet id="{}">{}</tweet>\n'.format(ID,Text))
            
        writeData.write('</tweets>\n')
    writeData.write('</data>')

## Loading XML file created

In [4]:
# importing xmltodict library to check if created xml file is laodable or not 
import xmltodict

with open('30749476.xml', 'r', encoding = 'utf-8') as fd:
    doc = xmltodict.parse(fd.read())